# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
df_binary = pd.get_dummies(df, columns=["homeowner"])
df_binary

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
77531,19100,11.261,86600,0.653580,12,2,56600,high_risk,0,1,0
77532,17700,10.662,80900,0.629172,11,2,50900,high_risk,1,0,0
77533,17600,10.595,80300,0.626401,11,2,50300,high_risk,0,0,1
77534,16300,10.068,75300,0.601594,10,2,45300,high_risk,1,0,0


In [5]:
# Create our features
X = df_binary.drop(columns="loan_status", axis=1)
# Create our target
y = df["loan_status"]

In [6]:
y.describe()

count        77536
unique           2
top       low_risk
freq         75036
Name: loan_status, dtype: object

In [7]:
y

0         low_risk
1         low_risk
2         low_risk
3         low_risk
4         low_risk
           ...    
77531    high_risk
77532    high_risk
77533    high_risk
77534    high_risk
77535    high_risk
Name: loan_status, Length: 77536, dtype: object

In [8]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
Counter(y_train)

Counter({'low_risk': 56271, 'high_risk': 1881})

In [12]:
X_train

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
29175,8600,6.792,44500,0.325843,3,0,14500,0,1,0
23020,7800,6.419,41000,0.268293,2,0,11000,0,1,0
31269,10000,7.386,50100,0.401198,4,1,20100,0,1,0
35479,9300,7.093,47300,0.365751,3,0,17300,0,0,1
13470,9200,7.045,46900,0.360341,3,0,16900,0,1,0
...,...,...,...,...,...,...,...,...,...,...
20609,7200,6.177,38700,0.224806,1,0,8700,0,1,0
21440,10000,7.389,50100,0.401198,4,1,20100,0,1,0
73349,10200,7.463,50800,0.409449,4,1,20800,0,1,0
50057,11100,7.838,54400,0.448529,5,1,24400,1,0,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train)

StandardScaler()

In [15]:
# Scale the training and testing data
X_train_scaled = data_scaler.transform(X_train)
X_test_scaled = data_scaler.transform(X_test)

# Simple Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]])

In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [20]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [21]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [22]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9936781215845847

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  116, 18649]])

In [24]:
# Print the imbalanced classification report
classification_report_imbalanced(y_test, y_pred)

'                   pre       rec       spe        f1       geo       iba       sup\n\n  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619\n   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765\n\navg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384\n'

### SMOTE Oversampling

In [25]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [26]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [27]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9936781215845847

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  116, 18649]])

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculate the balanced accuracy score

balanced_accuracy_score(y_test, y_pred)

0.9936781215845847

In [33]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  608,    11],
       [  112, 18653]])

In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [35]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
# View the count of target classes with Counter

Counter(y_resampled)

Counter({'high_risk': 55451, 'low_risk': 55938})

In [36]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [37]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9881304218875842

In [38]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  122, 18643]])

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?
   
   The best balanced accuracy score was shared by 2 models,
   SMOTE Oversampling had a score of 0.9936781215845847
   ClusterCentroids had a score of   0.9936781215845847

2. Which model had the best recall score?

    SMOTE and SMOTEENN shared the best recall scores of 0.99 for both low and high risk loans.

3. Which model had the best geometric mean score?

    ClusterCentroids, SMOTE and SMOTEENN all had a geometric mean score of 0.99
